In [1]:
from typing import List, Dict, Literal, Union, Callable
import datetime, time 
import numpy as np 
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

#import RPi.GPIO as GPIO
# from multiprocessing import Pipe, Process
# from multiprocessing.connection import Connection


from data_collection.data_collection import Logger, LoggerSet

from components.two_wheels import TwoWheels
from components.microphone.ahhhhh import  AhhhhhWheelController
from components.camera import Picamera

%load_ext autoreload
%autoreload 2

In [2]:
logger_set = LoggerSet('../log/test',  overwrite_ok=False)

In [3]:
cam_p = Picamera.start((114, 64), 25, {}, logger_set=logger_set, name='test12')

[2:06:04.620638187] [13267]  INFO Camera camera_manager.cpp:297 libcamera v0.0.5+83-bde9b04f


[2:06:04.664764769] [13268]  INFO RPI vc4.cpp:437 Registered camera /base/soc/i2c0mux/i2c@1/ov5647@36 to Unicam device /dev/media3 and ISP device /dev/media0
[2:06:04.665230913] [13268]  INFO RPI pipeline_base.cpp:1101 Using configuration file '/usr/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'
[2:06:04.676336669] [13267]  INFO Camera camera.cpp:1033 configuring streams: (0) 114x64-RGB888 (1) 1920x1080-SGRBG10_CSI2P
[2:06:04.678204579] [13268]  INFO RPI vc4.cpp:565 Sensor: /base/soc/i2c0mux/i2c@1/ov5647@36 - Selected sensor format: 1920x1080-SGRBG10_1X10 - Selected unicam format: 1920x1080-pgAA


In [16]:
cam_p.terminate()

In [6]:
logger_set.export_to_parquet()

100%|██████████| 1/1 [00:00<00:00,  3.76it/s]


{}

In [6]:
tmp = logger_set.get_all_logger()[0]

In [63]:
data = tmp.load()

In [74]:
df = tmp.load_as_df()

In [77]:
df.to_parquet('data.parquet')

In [78]:
df_loaded = pd.read_parquet('data.parquet')

In [79]:
df_loaded

key,AhhhhhDetector,AhhhhhWheelController,balance,basefreq,latest_sxx,median_pitch,pitch,pitch_power,speed_or_invalid
idx,,,,,,,,,
1,2024-02-24 22:20:50.012290,2024-02-24 22:20:50.015540,0.292740,2666.666667,"[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -in...",137.095842,137.095842,NaN,0.0
2,2024-02-24 22:20:50.031514,2024-02-24 22:20:50.032497,0.403040,98.765432,"[24.02644370069683, 23.975647587074846, 23.820...",181.216092,225.336341,39167.132280,0.0
3,2024-02-24 22:20:50.095978,2024-02-24 22:20:50.097486,0.292740,258.064516,"[25.46121177825205, 25.129861772779567, 23.933...",137.095842,4.171671,405418.776525,0.0
4,2024-02-24 22:20:50.160765,2024-02-24 22:20:50.161851,0.435568,93.023256,"[26.377028695693024, 25.368752229628825, 21.04...",194.227031,194.227031,115122.692343,0.0
5,2024-02-24 22:20:50.165562,2024-02-24 22:20:50.167135,0.800000,275.862069,"[27.024186669612714, 24.70830698240676, 24.182...",38.809184,38.809184,196477.148239,0.0
...,...,...,...,...,...,...,...,...,...
4748,2024-02-24 22:26:25.015593,2024-02-24 22:26:25.016539,0.684433,112.676056,"[27.627798521042997, 17.00999544049359, 19.437...",293.773379,293.773379,27870.830710,0.0
4749,2024-02-24 22:26:25.081477,2024-02-24 22:26:25.082375,0.684433,2666.666667,"[27.70662399038205, 20.831132078802394, 21.054...",293.773379,137.095842,0.000000,0.0
4750,2024-02-24 22:26:25.145242,2024-02-24 22:26:25.146146,0.292740,2666.666667,"[27.640918585504387, 13.897895598337326, 17.49...",137.095842,137.095842,0.000000,0.0


In [50]:
from pathlib import Path
class NormaliseRecord:
    type2normaliser = {
        np.ndarray: lambda x: x.tolist(), 
        datetime.datetime: str
    }

    def __init__(self, records:List, dir_path:Union[str, Path], name:str):
        # meant to be shared with other records
        dir_path = Path(dir_path)
        dir_path.mkdir(exist_ok=True)

        self.records = records
        self.dir_path = dir_path
        self.name = name
    
    def transform(self, key:str, func:Callable):
        for r in self.records:
            if r['key'] == key:
                r['value'] = func(r['value'])

    def datetime_to_str(self, key):
        self.transform(key, str)

    def numpy_to_list(self, key):
        self.transform(key, lambda v : v.tolist())

    def auto(self):
        for r in self.records: 
            value_type = type(r['value'])
            if value_type in self.type2normaliser:
                r['value'] = self.type2normaliser[value_type](r['value'])

    def save_to_json(self):
        with open(self.dir_path/f'{self.name}.json', 'w') as fp:
            json.dump(fp=fp, obj=self.records)
    
    def save_to_json_zipped(self):

        with gzip.open('data.json.gz', 'wt', encoding='UTF-8') as zipfile:
            json.dump(self.records, zipfile)



In [33]:
import gzip

In [52]:
data = NormaliseRecord(data, 'temp', 'test')

In [45]:
data.transform('latest_sxx', str)

In [54]:
data.auto()

In [66]:
def encoder(obj):
    if isinstance(obj, datetime.datetime):
        return {'__encoded_from': 'datetime.datetime', 'json-compatible': obj.strftime("%Y%m%dT%H:%M:%S.%f")}

    elif isinstance(obj, np.ndarray):
        return {'__encoded_from': 'np.ndarray', 'json-compatible': obj.tolist()}
    
    else:
        return obj

def decoder(obj):

    if not '__encoded_from' in obj:
        return obj

    if obj['__encoded_from'] == 'datetime.datetime':
        return datetime.datetime.strptime(obj["json-compatible"], "%Y%m%dT%H:%M:%S.%f")

    elif obj['__encoded_from'] == 'np.ndarray':
        return np.array(obj["json-compatible"])

    else:
        raise ValueError('unknown type')


def save_to_msgpack(data, path, encoder=encoder):
    packed_data = msgpack.packb(data, default = encoder)
    with open(path, 'wb') as file:
        file.write(packed_data)

def load_from_msgpack(path, decoder=decoder):

    with open(path, 'rb') as file:
        byte_data = file.read()

    # Deserialize the MessagePack byte string back to a Python object
    return msgpack.unpackb(byte_data, object_hook=decoder)



In [67]:
import msgpack

# Your dictionary to serialize

# Serialize the dictionary to a MessagePack byte string
packed_data = msgpack.packb(data, default= encoder)
 
# If you want to save this to a file
with open('data.msgpack', 'wb') as file:
    file.write(packed_data)

In [71]:
with open('data.msgpack', 'rb') as file:
    byte_data = file.read()

# Deserialize the MessagePack byte string back to a Python object
data_loaded = msgpack.unpackb(byte_data, object_hook=decoder)

In [72]:
data_loaded[1]['value']

array([-inf, -inf, -inf, ..., -inf, -inf, -inf])

In [57]:
sum([ 7901267,
10608854,
 9793782,
 9826225,
 9826275,
 9825125,
 9809879,
 9827415,]
)/1e6

77.418822

In [41]:
data.save_to_json_zipped()

KeyboardInterrupt: 

In [46]:
s  = json.dumps(data.records)

In [32]:
import json
data.save_to_json()

In [1]:
isinstance?

Signature: isinstance(obj, class_or_tuple, /)
Docstring:
Return whether an object is an instance of a class or of a subclass thereof.

A tuple, as in ``isinstance(x, (A, B, ...))``, may be given as the target to
check against. This is equivalent to ``isinstance(x, A) or isinstance(x, B)
or ...`` etc.
Type:      builtin_function_or_method

In [15]:
str(datetime.datetime.now())

'2024-02-23 16:26:33.818222'

In [18]:
arr = data[1]['value']

In [ ]:
np.savez(arr)

In [19]:
np.savez?

Signature:       np.savez(file, *args, **kwds)
Call signature:  np.savez(*args, **kwargs)
Type:            _ArrayFunctionDispatcher
String form:     <function savez at 0x7f845ae040>
File:            ~/projects/car1/.venv/lib/python3.9/site-packages/numpy/lib/npyio.py
Docstring:      
Save several arrays into a single file in uncompressed ``.npz`` format.

Provide arrays as keyword arguments to store them under the
corresponding name in the output file: ``savez(fn, x=x, y=y)``.

If arrays are specified as positional arguments, i.e., ``savez(fn,
x, y)``, their names will be `arr_0`, `arr_1`, etc.

Parameters
----------
file : str or file
    Either the filename (string) or an open file (file-like object)
    where the data will be saved. If file is a string or a Path, the
    ``.npz`` extension will be appended to the filename if it is not
    already there.
args : Arguments, optional
    Arrays to save to the file. Please use keyword arguments (see
    `kwds` below) to assign names to a

In [22]:

np.save('test', arr, allow_pickle=False)

In [25]:
def time_func(func, n):
    start_time = time.monotonic()
    for _ in range(n):
        func() 
    end_time = time.monotonic() - start_time
    return end_time

time_func(lambda: np.save('test', arr, allow_pickle=False), 100)

5.815982090000034


5.815982090000034

In [40]:
from PIL import Image

def img_to_jpg():
    img = Image.fromarray(arr)
    #img.save('test.png')
    img.save('test.jpg', 'JPEG')

def img_to_png():
    img = Image.fromarray(arr)
    img.save('test.png')

def img_to_bmp():
    img = Image.fromarray(arr)
    img.save('test.bmp', 'BMP')


In [29]:
img_to_png()

In [39]:
def time_func(func, n):
    start_time = time.monotonic()
    for _ in range(n):
        func() 
    end_time = time.monotonic() - start_time
    return end_time

time_func(lambda: np.save('test', arr, allow_pickle=False), 10)

0.9113273349994415

In [42]:
time_func(img_to_jpg, 10)

0.06740459499997087